# ==============================================================================
# FLUXO COMPLETO: PREPARAÇÃO E VALIDAÇÃO DE DADOS
# Machine Learning - FCUP
# ==============================================================================

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Ferramentas do Scikit-Learn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier # Modelo simples para testar

In [ ]:
# Configuração visual
sns.set_theme(style="whitegrid")

In [ ]:
# ------------------------------------------------------------------------------
# 1. CARREGAMENTO DOS DADOS
# ------------------------------------------------------------------------------
print("--- 1. Carregando Dataset Titanic ---")
df = pd.read_csv("Titanic-Dataset.csv")

# Para simplificar o exercício, vamos selecionar apenas colunas numéricas relevantes
# e a variável alvo 'survived'.
colunas_interesse = ['Survived', 'Pclass', 'Age', 'Fare']
df = df[colunas_interesse]

display(df.head())

In [ ]:
# ------------------------------------------------------------------------------
# 2. ANÁLISE BÁSICA (DIAGNÓSTICO)
# ------------------------------------------------------------------------------
print("\n--- 2. Diagnóstico: Dados em Falta e Escalas ---")

# A. Verificando dados em falta
print("Contagem de nulos por coluna:")
print(df.isnull().sum())
print("-" * 30)

In [ ]:
# B. Verificando Problemas de Escala
# Vamos comparar 'age' (Idade) e 'fare' (Tarifa)
print("Estatísticas Descritivas:")
display(df[['Age', 'Fare']].describe())

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
sns.boxplot(data=df[['Age', 'Fare']])
plt.title("Antes: Escalas Diferentes")
plt.xlabel("Variáveis")
plt.ylabel("Valor")

In [ ]:
# ------------------------------------------------------------------------------
# 3. TRATAMENTO: IMPUTAÇÃO (Preencher Nulos)
# ------------------------------------------------------------------------------
print("\n--- 3. Aplicação de Imputação ---")

# Estratégia: Como 'age' pode ter outliers, vamos usar a MEDIANA em vez da Média.
imputer = SimpleImputer(strategy='median')

# Nota: O fit_transform retorna um array numpy, precisamos converter de volta para DataFrame
# Aplicamos apenas na coluna 'age' que tinha falhas
df['Age'] = imputer.fit_transform(df[['Age']])

print("Nulos após imputação:")
print(df.isnull().sum())

In [ ]:
# ------------------------------------------------------------------------------
# 4. TRATAMENTO: NORMALIZAÇÃO (Ajustar Escalas)
# ------------------------------------------------------------------------------
print("\n--- 4. Aplicação de Normalização (Padronização) ---")

# Vamos usar Z-Score (StandardScaler) para colocar Idade e Tarifa na mesma régua.
scaler = StandardScaler()

# Salvamos uma cópia para não perder os dados originais se precisarmos
df_scaled = df.copy()

# Normalizamos as colunas numéricas contínuas
colunas_para_normalizar = ['Age', 'Fare']
df_scaled[colunas_para_normalizar] = scaler.fit_transform(df[colunas_para_normalizar])

# Visualizando o resultado da normalização
plt.subplot(1, 2, 2)
sns.boxplot(data=df_scaled[colunas_para_normalizar])
plt.title("Depois: Normalizado (Z-Score)")
plt.xlabel("Variáveis")
plt.ylabel("Desvios Padrão (Z)")

plt.tight_layout()
plt.show()

In [ ]:
# ------------------------------------------------------------------------------
# 5. DIVISÃO EM TREINO E TESTE (Holdout)
# ------------------------------------------------------------------------------
print("\n--- 5. Divisão Treino/Teste ---")

# Separar Atributos (X) do Alvo (y)
X = df_scaled.drop('Survived', axis=1) # Tudo menos 'survived'
y = df_scaled['Survived']              # Apenas 'survived'

# Divisão 70% Treino, 30% Teste
# stratify=y garante que a proporção de sobreviventes seja igual nos dois grupos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(f"Tamanho do Treino: {X_train.shape}")
print(f"Tamanho do Teste:  {X_test.shape}")

In [ ]:
# ------------------------------------------------------------------------------
# 6. VALIDAÇÃO CRUZADA (Cross-Validation)
# ------------------------------------------------------------------------------
print("\n--- 6. Validação Cruzada (k-Fold) ---")

# Vamos usar um modelo k-NN (Vizinhos Próximos)
# O k-NN depende MUITO da normalização que fizemos no passo 4.
modelo_knn = KNeighborsClassifier(n_neighbors=5)

# Aplicamos cross-validation com 5 pastas (folds) apenas nos dados de TREINO
scores = cross_val_score(modelo_knn, X_train, y_train, cv=5, scoring='accuracy')

print(f"Resultados das 5 iterações: {scores}")
print(f"Acurácia Média: {scores.mean():.2%}")
print(f"Desvio Padrão:  {scores.std():.2%}")

print("\n--- Conclusão ---")
print("O dataset foi limpo, normalizado e validado.")
print(f"Podemos esperar que este modelo acerte aprox. {scores.mean()*100:.1f}% das vezes em novos dados.")